In [138]:
import requests
from bs4 import BeautifulSoup
import re
import os
from pathlib import Path

from tqdm.notebook import tqdm

In [80]:
def make_dir(dir_path):
    if not dir_path.is_dir():
        os.mkdir(dir_path)
        print(f'Created: {dir_path.as_posix()}')
    else:
        print(f"{dir_path.as_posix()} exists")

In [146]:
def get_txt_script(script_url):
    '''
    Input is just the html page, without action added.
    return the text of the episode script. 
    Always starts with 5 dashes. And ends with 5 dashes
    Scenes are separated by also 5 dashes
    '''
    raw_html = requests.get(script_url)

    soup = BeautifulSoup(raw_html.text, 'html.parser')

    # Get the (one) box of the script links information
    try:
        main_table = soup.find_all('table', class_='headerscontent')[1]
    except IndexError as e:
        print(e)
        print(script_url)
        main_table = soup.find_all('table', class_='headerscontent')     
        text = ''
        for table in main_table:
            mids = main_table.find_all('td', class_='DLborderBOT')
            
            for mid in mids:
                if len(mid.text)<=2:
                    text += '-'*5 + '\n'
                text += mid.text + '\n'
            text += '-'*5

        return text
        
    
#     for table in main_table[1:]: # From table 1
#     rights = table.find_all('td', class_='DLborderRIGHT')
    mids = main_table.find_all('td', class_='DLborderBOT')

    text = ''
    for mid in mids:
        if len(mid.text)<=2:
            text += '-'*5 + '\n'
        text += mid.text + '\n'
    text += '-'*5

    return text

Create the directory to store all the scripts in.

In [82]:
current_dir = Path.cwd() # So that I know where I start

# Set the path of the directory in which the scripts for each season and each episode will be stored in
scripts_path = current_dir / 'Scripts'

# Create the Scripts dir
make_dir(scripts_path)

C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts exists


Now I can get the links to all the seasons to search in for episodes and finally scripts.

Url of the [scripts portal](https://southpark.fandom.com/wiki/Portal:Scripts). Lists all the seasons.

In [91]:
# Set the base url. Used to add to it
base_url = r'https://southpark.fandom.com/wiki/'

# For the 'API', when we want to get the wikisource style page. It is easier to parse
action = r'?action=edit' 

# Create the url to get the seasons links
scripts_main_url = base_url+ r'Portal:Scripts' + action
raw_scripts_main_html = requests.get(scripts_main_url)
soup_season = BeautifulSoup(raw_scripts_main_html.text, 'html.parser')

# Get the (one) box of the script links information
main_box_seasons = soup_season.find_all('textarea', id='wpTextbox1')[0] # It always has 1 box

In [116]:
# Use regex to get all the seasons links in a list. This pattern will be used
# for the seasons links and then the episodes links.
# pattern = r'\|link\=(.+)\n'

pattern = r"\|link\=([^\|\n]+)\n"

# Create a dictionary for the links. It is a dict of dicts. Keys are season numbers and values are dictionaries with keys episode
# and link as value
scripts_dict = {}

Now start going inside every season then every episode and finally inside each script. Get its text and store it in a text file in the corresponding folder.

In [149]:
season = 1
matches_season = re.findall(pattern , main_box_seasons.get_text())
for match_season in tqdm(matches_season):
    # Get the link and replace space with _. Not essential, automatically done
    link_name_seas = match_season.replace(' ', '_')
    
    # Get the name of the season 
#     ses_dir_name = link_name.split('/')[-1]
    ses_dir_name = str(season)
    
    # Directory to create
    season_dir_path = scripts_path / ses_dir_name
    make_dir(season_dir_path)
    
    ## Get all the episodes in this season. Similar procedure
    episodes_url = base_url + link_name_seas + action
    episodes_html = requests.get(episodes_url)
    soup_episodes = BeautifulSoup(episodes_html.text, 'html.parser')
    
    # Get the (one) box of the script links information
    main_box_episodes = soup_episodes.find_all('textarea', id='wpTextbox1')[0] # It always has 1 box
    episode = 1
    matches_episodes = re.findall(pattern , main_box_episodes.get_text())
        
    for match_episode in matches_episodes:
        # Get the link and replace space with _. Not essential, automatically done
        link_name_ep = match_episode.replace(' ', '_')
                
        # Get the name of the episode. Add its number in the beginning 
        fname = link_name_ep.split('/')[-2].replace('?', '').replace('\'', '').replace('.', '') # It has script added in the end
        fname = fname.replace(':', "_")
        ep_dir_name = str(episode) + "_" + fname + '.txt' 
        
        # Directory to create
        episode_dir_path = season_dir_path / ep_dir_name
        Path(episode_dir_path).touch()
        
        
        script_link = base_url + link_name_ep
        with open(episode_dir_path.as_posix(), 'w', encoding='utf-8') as f:
            f.write(get_txt_script(script_link.replace('?', '%3F')))
        
        
        # Increment episode
        episode += 1
        
    
        # Update dict
        scripts_dict[season] = {episode: script_link}
    
    # Increment season number
    season += 1

  0%|          | 0/25 [00:00<?, ?it/s]

Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/1
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/2
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/3
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/4
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/5
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/6
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/7
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/8
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_

In [9]:
boxes[0]

<textarea accesskey="," class="mw-editfont-default" cols="80" dir="ltr" id="wpTextbox1" lang="en" name="wpTextbox1" readonly="" rows="25" style="">{{TabPortalScripts}}
This portal is to help you search for scripts of episodes.

==Scripts by Season==
&lt;gallery widths="140" captionalign="center" navigation="true"&gt;
Season1.jpg|[[Portal:Scripts/Season One|Season One]]|link=Portal:Scripts/Season One
Season2.jpg|[[Portal:Scripts/Season Two|Season Two]]|link=Portal:Scripts/Season Two
Season3.jpg|[[Portal:Scripts/Season Three|Season Three]]|link=Portal:Scripts/Season Three
Season4.jpg|[[Portal:Scripts/Season Four|Season Four]]|link=Portal:Scripts/Season Four
Season5.jpg|[[Portal:Scripts/Season Five|Season Five]]|link=Portal:Scripts/Season Five
Season6.jpg|[[Portal:Scripts/Season Six|Season Six]]|link=Portal:Scripts/Season Six
Season7.jpg|[[Portal:Scripts/Season Seven|Season Seven]]|link=Portal:Scripts/Season Seven
Season8.jpg|[[Portal:Scripts/Season Eight|Season Eight]]|link=Portal:Script

Initially, I want to get the links to all the episodes' scripts. I can use the action to the edit page from the main scripts page.

In [39]:
scripts_main_url = base_url+ r'Portal:Scripts' + action
raw_scripts_main_html = requests.get(scripts_main_url)

soup = BeautifulSoup(raw_scripts_main_html.text, 'html.parser')

# Get the (one) box of the script links information
main_box = soup.find_all('textarea', id='wpTextbox1')[0] # It is only 1 always

In [40]:
# Use regex to get all the seasons links in a list.

pattern = r'\|link\=(.+)\n'

seasons_links = []

matches = re.findall(pattern , main_box.get_text())
for match in matches:
    name_ = match.replace(' ', '_')
    seasons_links.append(base_url+name_+action)

In [41]:
for link in seasons_links:
    print(link)

https://southpark.fandom.com/wiki/Portal:Scripts/Season_One?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Two?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Three?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Four?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Five?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Six?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Seven?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Eight?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Nine?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Ten?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Eleven?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Twelve?action=edit
https://southpark.fandom.com/wiki/Portal:Scripts/Season_Thirteen?action=edit
https://southpark.fandom.com/wiki/Portal:Scr

As a test, lets get all the episodes script links for season 1

In [131]:
season_url = seasons_links[2]

raw_scripts_episodes_html = requests.get(season_url)

soup = BeautifulSoup(raw_scripts_episodes_html.text, 'html.parser')

# Get the (one) box of the script links information
main_box = soup.find_all('textarea', id='wpTextbox1')[0] # It is only 1 always

In [132]:
matches = re.findall(pattern , main_box.text)
for match in matches:
    name_ = match.replace(' ', '_')
    print(base_url+name_)

https://southpark.fandom.com/wiki/Rainforest_Shmainforest/Script
https://southpark.fandom.com/wiki/Spontaneous_Combustion/Script
https://southpark.fandom.com/wiki/The_Succubus/Script
https://southpark.fandom.com/wiki/Jakovasaurs/Script
https://southpark.fandom.com/wiki/Tweek_vs._Craig/Script
https://southpark.fandom.com/wiki/Sexual_Harassment_Panda/Script
https://southpark.fandom.com/wiki/Cat_Orgy/Script
https://southpark.fandom.com/wiki/Two_Guys_Naked_in_a_Hot_Tub/Script
https://southpark.fandom.com/wiki/Jewbilee/Script
https://southpark.fandom.com/wiki/Korn's_Groovy_Pirate_Ghost_Mystery/Script
https://southpark.fandom.com/wiki/Chinpokomon/Script
https://southpark.fandom.com/wiki/Hooked_on_Monkey_Fonics/Script
https://southpark.fandom.com/wiki/Starvin'_Marvin_in_Space/Script
https://southpark.fandom.com/wiki/The_Red_Badge_of_Gayness/Script
https://southpark.fandom.com/wiki/Mr._Hankey's_Christmas_Classics/Script
https://southpark.fandom.com/wiki/Are_You_There_God?_It's_Me,_Jesus/Script

In [134]:
raw_html = requests.get(r"https://southpark.fandom.com/wiki/Are_You_There_God?_It's_Me,_Jesus/Script")

soup = BeautifulSoup(raw_html.text, 'html.parser')

# Get the (one) box of the script links information
main_table = soup.find_all('table', class_='headerscontent')

In [136]:
for table in main_table[0:]: # From table 1
    rights = table.find_all('td', class_='DLborderRIGHT')
    mids = table.find_all('td', class_='DLborderBOT')
    for mid in mids:
        if len(mid.text)<=2:
            print('-'*50)
        print(mid.text)
    print('-'*50)   
    

In [126]:
print(get_txt_script(r"https://southpark.fandom.com/wiki/Wing/Script"))

-----


South Park Elementary, day. Class is in session. The kids are chattering when Mrs. Garrison gets their attention.

Mrs. Garrison

Children, children, we have a very special announcement. [shown with Tolkien at her side] Your classmate, Tolkien, has just returned from winning the Colorado Child Star Contest with his incredible singing! Well let's give him a hand! [She claps, and Butters, Annie, and Bebe join him]

Stan

Token sings?

Cartman

Of course he sings. He's black.

Mrs. Garrison

Tolkien is going to sing in front of a huge audience at the Miss Colorado Pageant in Denver. And for doing it he's gonna be paid two hundred dollars.

Class

Whoa!

Cartman

Two hundred dollars.

Mrs. Garrison

Now of course, as a woman, I'm a bit against those chauvinistic pageants, but I hope that you children take a moment today to congratulate Tolkien for being so special.

-----


South Park Elementary, lunchtime. Stan and friends sit at a table just outside the school kitchen. They're de